In [1]:
import time
import torch 
from torch import nn,optim

import d2l_pytorch as d2l 

import sys
sys.path.append('..')

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def ftn_blk(num_convs,in_channels,out_channels,dilation):
    blk=[]
    for i in range(num_convs):
        # 卷积块中的第一个卷积层
        if i ==0:
            if dilation == 0:
                blk.append(nn.Conv2d(in_channels,out_channels,kernel_size=3))
            else:
                blk.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,dilation=dilation))
        else:
            if dilation == 0:
                blk.append(nn.Conv2d(out_channels,out_channels,kernel_size=3))
            else:
                blk.append(nn.Conv2d(out_channels,out_channels,kernel_size=3,dilation=dilation))
        blk.append(nn.ReLU(inplace=True))
    return blk

In [3]:
def frontend_vgg(in_channels,num_class):
    conv_arch=(
        (2,in_channels, 64),
        (2, 64,128),
        (3,128,256),
        (3,256,512),
        (3,512,512)
    ) 
    #  (conv nums of blk,in_channels,out_channels) 
    
    dilations = [0, 0, 0, 0, 2, 4]
    # dilation factor
    
    net=nn.Sequential()

    for l,(convs_num,in_channels,out_channels) in enumerate(conv_arch):
        blk=ftn_blk(convs_num,in_channels,out_channels,dilations[l])
        if dilations[l+1]==0:
            blk.append(nn.MaxPool2d(kernel_size=2,stride=2))
        net.add_module('ftn_blk_'+str(l+1),nn.Sequential(*blk))
        
    # layer_6
    net.add_module('ftn_fc6',nn.Sequential(
            nn.Conv2d(out_channels,4096,kernel_size=7,dilation=dilations[5]),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5))
    )
    # layer_7
    net.add_module('ftn_fc7',nn.Sequential(
        nn.Conv2d(4096,4096,kernel_size=1),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5))
    )
    # layer_8
    net.add_module('ftn_fc8',nn.Conv2d(4096,num_class,kernel_size=1))

    return net

In [4]:
# test

fn=frontend_vgg(3,10)
print(fn)

Sequential(
  (ftn_blk_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ftn_blk_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ftn_blk_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (ftn_blk_4): Sequential(